# **워드 임베딩 + 클러스터링**

In [1]:
# 기본 패키지 불러오기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# 데이터 파일 불러오기
df_1 = pd.read_csv('book.csv', encoding='euc-kr')
# df_1.head()

In [3]:
df_1.tail()

,Unnamed: 0,ID,Title,Rating,Author,Price,Pdate,Publisher,Btype,Salseindex
33298,33298,97039591,뇌 1,NaN,베르나르 베르베르,"4,950",2006년 04월 10일,열린책들,old,NaN
33299,33299,97039592,나의 문화유산답사기 6,NaN,유홍준,"6,930",2011년 05월 11일,창비,old,NaN
33300,33300,97039593,말테의 수기,NaN,라이너 마리아 릴케,"3,780",2005년 01월 15일,민음사,old,NaN
33301,33301,97039594,다빈치 코드 1,NaN,댄 브라운,"5,040",2013년 12월 11일,문학수첩,old,NaN
33302,33302,97039595,밥상에 오른 과학,NaN,이성규,"3,990",2007년 05월 20일,봄나무,old,NaN


In [4]:
# 전처리 - price  object => 수치형
# df['Price'] = pd.to_numeric(df['Price'].str.replace('[^\d.]', ''), errors='coerce')

# 'Pdate' 컬럼을 날짜 타입으로 변환 (한국어 날짜 형식에 맞춤)
# df['Pdate'] = pd.to_datetime(df['Pdate'], format='%Y년 %m월 %d일', errors='coerce')


In [5]:
# 전처리 - 공백 제거
df_1 = df_1.dropna(subset=['Title'])
df_1 = df_1[df_1['Title'].str.strip() != '']

# df_2 = df_1.dropna(subset=['Publisher'])
# df_2 = df_2[df_2['Publisher'].str.strip() != '']

In [6]:
# 전처리 - 언어별 구분
# df = df_2[df_1['Title'].str.contains('[A-Za-z]', regex=True)]
# 한국어 [가-힣]
# 영어[A-Za-z]
# 일본어 [ぁ-んァ-ン]


In [7]:
# 전처리 - 언어별 구분
import re

# 영어 제목만
def is_english_title(text):
    # 영어 알파벳, 공백, 일부 특수문자(예: ',!?.)만 허용
    return bool(re.match(r'^[a-zA-Z0-9 .,\-\'!?]+$', text))

# 영어로만 구성된 책 제목만 가져옴
df_e = df_1[df_1['Title'].apply(is_english_title)]

# 한국 제목만
def is_korean_title(text):
    # 영어 알파벳, 공백, 일부 특수문자(예: ',!?.)만 허용
    return bool(re.match(r'^[가-힣0-9 .,\-\'!?]+$', text))

# 한국어로만 구성된 책 제목만 가져옴
df_k = df_1[df_1['Title'].apply(is_korean_title)]


## df_k,   df_e  변수명 변경 시... 관련된 부분 모두 수정해 주어야 함

In [48]:
df_k

,Unnamed: 0,ID,Title,Rating,Author,Price,Pdate,Publisher,Btype,Salseindex,Cluster
12,12,97006297,강철왕국 프로이센,NaN,크리스토퍼 클라크,"41,150",2020년 07월 20일,마티,old,NaN,0
24,24,97006309,로마인 이야기 1-15권 세트,NaN,시오노 나나미,"138,800",2007년 02월 05일,한길사,old,NaN,8
25,25,97006310,이화림 회고록,NaN,이화림,"80,000",2015년 03월 26일,차이나하우스,old,NaN,9
66,66,97006351,쓰레기에 관한 모든 것,NaN,피에로 마르틴,"17,090",2020년 07월 15일,북스힐,old,NaN,9
90,90,97006375,고전 기하학과 현대 기하학,NaN,김의철,"20,890",2020년 07월 30일,미지현,old,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...
33298,33298,97039591,뇌 1,NaN,베르나르 베르베르,"4,950",2006년 04월 10일,열린책들,old,NaN,5
33299,33299,97039592,나의 문화유산답사기 6,NaN,유홍준,"6,930",2011년 05월 11일,창비,old,NaN,8
33300,33300,97039593,말테의 수기,NaN,라이너 마리아 릴케,"3,780",2005년 01월 15일,민음사,old,NaN,9
33301,33301,97039594,다빈치 코드 1,NaN,댄 브라운,"5,040",2013년 12월 11일,문학수첩,old,NaN,5


In [8]:
# 벡터화 + 클러스터링을 위한 패키지
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans


**Tf-idf** <br>
단어의 출현 빈도 및 상대 빈도 활용

In [47]:
tfidf_vectorizer = TfidfVectorizer(max_features=100) # 전체 단어 집합에서 TF-IDF 값이 가장 높은 상위 n개의 단어만을 선택하여 특성 벡터를 생성
X_tfidf = tfidf_vectorizer.fit_transform(df_k)

In [10]:
# 클러스터링 수행
kmeans = KMeans(n_clusters=10, random_state=42)
kmeans.fit(X_tfidf)

### 이후 아래 #123 코드 셀로 이동하여 코드 실행

KMeans(n_clusters=10, random_state=42)

**Word2Vec** <br>
주변단어들을 학습에 사용 <br> <br>
CBOW<br>
주변 단어들(문맥)-타겟 단어의 앞뒤에 위치한 단어들-을 기반으로 타겟 단어 예측<br>
ex "The cat sits on the ___" 빈칸에 들어갈 단어(타겟 단어) 예측<br><br>
Skip-gram<br>
특정 단어를 입력으로 받아, 그 단어 주변의 문맥 단어를 예측<br>
ex "cat"이라는 단어가 주어졌을 때, 이 단어 주변에 위치할 가능성이 높은 단어("The", "sits", "on") 예측

In [11]:
################# Word2Vec  #################
from gensim.models import Word2Vec
import pandas as pd
import numpy as np

# 책 제목을 단어 리스트로 변환
sentences = [title.split() for title in df_k['Title']]

# Word2Vec 모델 학습
word2vec_model = Word2Vec(sentences, vector_size=64, window=5, min_count=1)
   # 책 제목에서 각 단어의 벡터 표현 학습
   # 학습을 통해 책 제목을 구성하는 단어들 간의 관계와 문맥을 바탕으로 각 단어의 의미를 반영하는 벡터 생성


In [12]:
# 책 제목 -> 벡터값 조회 및 제목단위 평균값 산출
# 학습된 word2vec 모델을 사용하여 각 책 제목을 구성하는 단어들의 벡터 값 조회
def vectorize_w_word2vec(text):
    words = text.split()
    word_vectors = [word2vec_model.wv[word] for word in words if word in word2vec_model.wv]
    if len(word_vectors) == 0:
        return np.zeros(word2vec_model.vector_size)
    return np.mean(word_vectors, axis=0)

# 각 책 제목을 벡터화
title_vectors_k = np.array([vectorize_w_word2vec(title) for title in df_k['Title']])

### 이 후 아래 클러스터링 코드 실행하여 결과 확인

**FastText** <br>
주변 단어들을 학습에 사용 =  word2vec <br>
서브 워드 사용 <br>

In [13]:
################# FastText  #################
from gensim.models import FastText
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np

# FastText 모델 학습 (로컬 데이터)
# sentences = [title.split() for title in df_e['Title']]
sentences = [title.split() for title in df_k['Title']]
fasttext_model = FastText(sentences, vector_size=64, window=5, min_count=1)
   # 책 제목에서 각 단어와 서브워드(subword)의 벡터 표현 학습
   # 학습을 통해 책 제목을 구성하는 단어들 간의 관계와 문맥을 바탕으로 각 단어의 의미를 반영하는 벡터 생성

In [14]:
# 책 제목 -> 벡터값 조회 및 제목단위 평균값 산출
# 학습된 FastText 모델을 사용하여 각 책 제목을 구성하는 단어들의 벡터 값 조회
def vectorize_w_fasttext(text):
    words = text.split()
    word_vectors = [fasttext_model.wv[word] for word in words if word in fasttext_model.wv]
    if len(word_vectors) == 0:
        return np.zeros(fasttext_model.vector_size)
    return np.mean(word_vectors, axis=0)

# 제목 벡터화
title_vectors_k = np.array([vectorize_w_fasttext(title) for title in df_k['Title']])

**아래 클러스터링 과정은 동일**

In [15]:
# K-평균 클러스터링 수행
kmeans = KMeans(n_clusters=10, random_state=42)
kmeans.fit(title_vectors_k)

# 클러스터 할당 결과
df_k['Cluster'] = kmeans.labels_

C:\Users\crazy\AppData\Local\Temp\ipykernel_8756\4099749918.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_k['Cluster'] = kmeans.labels_


In [16]:
# 123 코드 셀
# 클러스터 별로 데이터 확인
for cluster in range(10): # 클러스터의 수에 따라 범위 조정
    print(f"Cluster {cluster}:")
    print(df_k[df_k['Cluster'] == cluster]['Title'].head(), '\n') # 각 클러스터에 속한 책 제목 출력


Cluster 0:
12               강철왕국 프로이센
126                 경영의 명의
262            가족신탁 이론과 실무
278    스피킹 매트릭스 30초 영어 말하기
344        생각하는 아이 기다리는 엄마
Name: Title, dtype: object 

Cluster 1:
1024           한국사 편지 세트
1724      설민석의 한국사 대모험 5
1819    특종! 70명으로 읽는 한국사
1948      설민석의 한국사 대모험 4
3719      이두호의 머털이 한국사 9
Name: Title, dtype: object 

Cluster 2:
1719     쿠키런 서바이벌 대작전 7
1738     쿠키런 서바이벌 대작전 2
1739     쿠키런 서바이벌 대작전 1
1740     쿠키런 서바이벌 대작전 3
1768    쿠키런 서바이벌 대작전 16
Name: Title, dtype: object 

Cluster 3:
1711         쿠키런 한자런 7
1733      쿠키런 개그 과학 상식
1743         쿠키런 한자런 8
1837    쿠키런 신대륙에서 찾아라!
1898         쿠키런 한자런 6
Name: Title, dtype: object 

Cluster 4:
90     고전 기하학과 현대 기하학
98           창의적 공학설계
153     응용이 보이는 선형대수학
173         알기쉬운 해부생리
266          내부감사 매뉴얼
Name: Title, dtype: object 

Cluster 5:
409    묻고 답하는 한국사카페 1
442        두근두근 중국어 1
476         런웨이의 연인 1
518     우리에게는 벽이 있다 1
529          비밀의 아이 4
Name: Title, dtype: object 

Cluster 6:
93         건축견적이야기 2 
357         

In [46]:
similar_words = fasttext_model.wv.most_similar('국민')

for word, similarity in similar_words:
    print(f"Word: {word}, similarity : {similarity}")

Word: 공황장애, similarity : 0.44372034072875977
Word: 동작교육과, similarity : 0.4418831169605255
Word: 일본, similarity : 0.4391671121120453
Word: 뉴노멀, similarity : 0.42190107703208923
Word: 레퍼런스, similarity : 0.42059946060180664
Word: 무너지지, similarity : 0.41250964999198914
Word: 좋았다, similarity : 0.4073355197906494
Word: 국민대차대조표, similarity : 0.40582984685897827
Word: 유럽만사, similarity : 0.40318357944488525
Word: 끼고, similarity : 0.4021202027797699


In [41]:
# 유사한 단어
fasttext_model.wv.most_similar('바바')

[('공주와', 0.5141741037368774),
 ('딥펜으로', 0.46176180243492126),
 ('문학,', 0.4553402066230774),
 ('수라도', 0.43647220730781555),
 ('맥의', 0.4217603802680969),
 ('있어야지', 0.4063689410686493),
 ('충동', 0.40417006611824036),
 ('24', 0.4018411934375763),
 ('거짓된', 0.3994196951389313),
 ('어디', 0.39838743209838867)]